In [ ]:
!pip install tensorflow keras opencv-python numpy matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from tensorflow.keras import optimizers, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':64,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
# 5. Prepare the training data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/carcrash/train.csv')
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [ ]:
df.video_path = df.video_path.str.replace('./train/', '/content/drive/MyDrive/Colab Notebooks/carcrash/train/')


In [ ]:
df

,sample_id,video_path,label
0,TRAIN_0000,/content/drive/MyDrive/Colab Notebooks/carcras...,7
1,TRAIN_0001,/content/drive/MyDrive/Colab Notebooks/carcras...,7
2,TRAIN_0002,/content/drive/MyDrive/Colab Notebooks/carcras...,0
3,TRAIN_0003,/content/drive/MyDrive/Colab Notebooks/carcras...,0
4,TRAIN_0004,/content/drive/MyDrive/Colab Notebooks/carcras...,1
...,...,...,...
2693,TRAIN_2693,/content/drive/MyDrive/Colab Notebooks/carcras...,3
2694,TRAIN_2694,/content/drive/MyDrive/Colab Notebooks/carcras...,5
2695,TRAIN_2695,/content/drive/MyDrive/Colab Notebooks/carcras...,0
2696,TRAIN_2696,/content/drive/MyDrive/Colab Notebooks/carcras...,0


In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

In [ ]:
def frames_from_video_file(video_path): # video를 읽어서 50fps에 해당하는 사진으로 반환
    frames = []
    cap = cv2.VideoCapture(video_path)
    for _ in range(CFG['VIDEO_LENGTH']):
        _, img = cap.read()
        img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        img = img / 255.
        frames.append(img)

    result = np.array(frames)[..., [2,1,0]]

    return result

class FrameGenerator: # 모든 video를 ram에 적재할 수 없기 때문에 batch 마다 generate 해주기 위한 generator 생성
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __call__(self):
        for tp in self.dataframe.itertuples():
            video_frames = frames_from_video_file(tp.video_path)
            label = tp.label # Encode labels
            label = to_categorical(label, 13)

            yield video_frames, label

In [ ]:
output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(13), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(train),
                                          output_signature=output_signature)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(val),
                                          output_signature=output_signature)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
train_ds = train_ds.batch(CFG['BATCH_SIZE'])
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.batch(CFG['BATCH_SIZE'])

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (4, 50, 64, 64, 3)
Shape of training labels: (4, 13)
Shape of validation set of frames: (4, 50, 64, 64, 3)
Shape of validation labels: (4, 13)


In [ ]:
# Define the video classification model:
model = Sequential()
model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(50, 64, 64, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(13, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(CFG['LEARNING_RATE']),
              metrics=['accuracy'])


In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 48, 62, 62, 32)    2624      
                                                                 
 batch_normalization (BatchN  (None, 48, 62, 62, 32)   128       
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 24, 31, 31, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 31, 31, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 22, 29, 29, 64)    55360     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 29, 29, 64)   2

In [ ]:
history = model.fit(train_ds, batch_size=10, epochs=50, verbose=1, validation_data=val_ds)


Epoch 1/50
540/540 [==============================] - 1017s 2s/step - loss: 2.3971 - accuracy: 0.3443 - val_loss: 1.7235 - val_accuracy: 0.5519
Epoch 2/50
540/540 [==============================] - 31s 58ms/step - loss: 1.6719 - accuracy: 0.5315 - val_loss: 1.4785 - val_accuracy: 0.6111
Epoch 3/50
540/540 [==============================] - 31s 58ms/step - loss: 1.3730 - accuracy: 0.6182 - val_loss: 1.4633 - val_accuracy: 0.6648
Epoch 4/50
540/540 [==============================] - 31s 58ms/step - loss: 1.2633 - accuracy: 0.6367 - val_loss: 0.9977 - val_accuracy: 0.7185
Epoch 5/50
540/540 [==============================] - 32s 58ms/step - loss: 1.1684 - accuracy: 0.6617 - val_loss: 0.9694 - val_accuracy: 0.7148
Epoch 6/50
540/540 [==============================] - 32s 58ms/step - loss: 1.1165 - accuracy: 0.6724 - val_loss: 0.9290 - val_accuracy: 0.7056
Epoch 7/50
540/540 [==============================] - 31s 58ms/step - loss: 1.0836 - accuracy: 0.6779 - val_loss: 0.9902 - val_accuracy:

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/carcrash/test.csv')


In [ ]:
test_df['label'] = 0
test_df.video_path = test_df.video_path.str.replace('./test/', '/content/drive/MyDrive/Colab Notebooks/carcrash/test/')


In [ ]:

test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_df),
                                          output_signature=output_signature)

test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.batch(CFG['BATCH_SIZE'])

pred = model.predict(test_ds)
pred = np.argmax(pred, axis=1)

submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/carcrash/sample_submission.csv')
submit['label'] = pred
submit.to_csv('./submission.csv', index=False)
print('pred and save done.')

    255/Unknown - 473s 2s/step